## `Twitter Farmer protests Sentiment analysis project-NLP`

In [70]:
!pip install textblob

Defaulting to user installation because normal site-packages is not writeable


In [52]:
import nltk
#nltk.download('stopwords')                            # Download Stopwords.
#nltk.download('punkt')
#nltk.download('wordnet')
 nltk.download('omw-1.4')

IndentationError: unexpected indent (2040762209.py, line 5)

In [6]:
# install and import necessary libraries.

import re, string, unicodedata                          # Import Regex, string and unicodedata.
import contractions                                     # Import contractions library.
from bs4 import BeautifulSoup                           # Import BeautifulSoup.

import numpy as np                                      # Import numpy.
import pandas as pd                                     # Import pandas.
import nltk                                             # Import Natural Language Tool-Kit.

# nltk.download('stopwords')                            # Download Stopwords.
# nltk.download('punkt')
# nltk.download('wordnet')

from nltk.corpus import stopwords   # Import stopwords.
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.
import matplotlib.pyplot as plt                         # Import plt for visualization

from pandas_profiling import ProfileReport              # library for creating a data profile

import matplotlib.pyplot as plt                         # low level visualization library
%matplotlib inline
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
import seaborn as sns            # higher level visualization library compared to matplotlib

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split        #Split data into training and testing set.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay 
.

import warnings                                         # Ignore Warnings 
#Suppress warnings
warnings.filterwarnings('ignore')

In [7]:
#Increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [8]:
#Load the data
data=pd.read_csv('tweets.csv')
#Verify the Shape of the dataset.
data.shape  

(1084452, 14)

In [9]:
#View the first 5 rows of the dataset.
data.head()   

,tweetUrl,date,renderedContent,tweetId,userId,replyCount,retweetCount,likeCount,quoteCount,source,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ShashiRajbhar6/status/1376...,2021-03-30 03:33:46+00:00,Support 👇\n\n#FarmersProtest,1.376739e+18,1.015970e+18,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN
1,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:33:23+00:00,Supporting farmers means supporting our countr...,1.376739e+18,1.332937e+18,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,NaN,NaN
2,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:31:00+00:00,Support farmers if you are related to food #St...,1.376739e+18,1.332937e+18,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,NaN,NaN
3,https://twitter.com/SukhdevSingh_/status/13767...,2021-03-30 03:30:45+00:00,#StopHateAgainstFarmers support #FarmersProtes...,1.376739e+18,1.308357e+18,0,1,3,0,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN
4,https://twitter.com/Davidmu66668113/status/137...,2021-03-30 03:30:30+00:00,"You hate farmers I hate you, \nif you love the...",1.376739e+18,1.357312e+18,0,0,1,0,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084452 entries, 0 to 1084451
Data columns (total 14 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   tweetUrl         1084452 non-null  object 
 1   date             1084452 non-null  object 
 2   renderedContent  1084452 non-null  object 
 3   tweetId          1084452 non-null  float64
 4   userId           1084452 non-null  float64
 5   replyCount       1084452 non-null  int64  
 6   retweetCount     1084452 non-null  int64  
 7   likeCount        1084452 non-null  int64  
 8   quoteCount       1084452 non-null  int64  
 9   source           1072318 non-null  object 
 10  media            319141 non-null   object 
 11  retweetedTweet   0 non-null        float64
 12  quotedTweet      286988 non-null   object 
 13  mentionedUsers   412206 non-null   object 
dtypes: float64(3), int64(4), object(7)
memory usage: 115.8+ MB


**Think about it:**

- We know that our dataset has 14640 rows of data and 15 columns, 
- By looking at the above output of data.info(), we can see that none of the columns are having null values. We can further confirm this using isna() method.**

In [11]:
#lets analysze the distribution of the various attribute
data.describe().T

,count,mean,std,min,25%,50%,75%,max
tweetId,1084452.0,1.379111e+18,3.551713e+16,1.322740e+18,1.356739e+18,1.362177e+18,1.412251e+18,1.462294e+18
userId,1084452.0,7.973759e+17,6.061264e+17,3.336000e+03,1.904166e+09,1.153479e+18,1.333602e+18,1.462105e+18
replyCount,1084452.0,1.010154e+00,1.699125e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.630650e+05
retweetCount,1084452.0,8.655195e+00,3.419062e+02,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,3.155470e+05
likeCount,1084452.0,1.869192e+01,1.045634e+03,0.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,9.443070e+05
quoteCount,1084452.0,6.005568e-01,4.889672e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.583200e+04
retweetedTweet,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Number of unique values in each column
data.nunique()

tweetUrl           1019415
date                922912
renderedContent     985573
tweetId             783132
userId              164601
replyCount             437
retweetCount          1449
likeCount             2665
quoteCount             399
source                 217
media               275822
retweetedTweet           0
quotedTweet         179906
mentionedUsers      155959
dtype: int64

**Insights:**
- `tweet_id column has 14485 unique values`

In [13]:
data.columns

Index(['tweetUrl', 'date', 'renderedContent', 'tweetId', 'userId',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'source',
       'media', 'retweetedTweet', 'quotedTweet', 'mentionedUsers'],
      dtype='object')

In [14]:
columns=['tweetUrl', 'date', 'tweetId', 'userId',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'source',
       'media', 'retweetedTweet', 'quotedTweet', 'mentionedUsers']
data.drop(columns=columns, inplace= True)

In [15]:
data.head()

,renderedContent
0,Support 👇\n\n#FarmersProtest
1,Supporting farmers means supporting our countr...
2,Support farmers if you are related to food #St...
3,#StopHateAgainstFarmers support #FarmersProtes...
4,"You hate farmers I hate you, \nif you love the..."


**Text Pre-Processing - Data Preparation**

In [16]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [17]:
data.renderedContent= data['renderedContent'].apply(to_lowercase)

In [18]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word =text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
        new_word = re.sub(r'\@w+|\#','',text)
        new_word = re.sub(r'[^\w\s]','',text) 
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [19]:
def data_processing(text):
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [20]:
data.renderedContent= data['renderedContent'].apply(data_processing)

TypeError: expected string or bytes-like object

In [ ]:
data=data.drop_duplicates('renderedContent')

In [ ]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

data['renderedContent'] = text_df['text'].apply(lambda x: stemming(x))

In [ ]:
data['renderedContent'] = text_df['renderedContent'].apply(lambda x: stemming(x))

In [ ]:
data.head()

In [ ]:
def polarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
data['polarity'] = data['text'].apply(polarity)

In [ ]:
text_df.head(10)

In [ ]:
def sentiment(label):
    if label <0:
        return "Negative"
    elif label ==0:
        return "Neutral"
    elif label>0:
        return "Positive"

In [ ]:
data['sentiment'] = data['polarity'].apply(sentiment)

In [ ]:
data.head()

In [ ]:
fig = plt.figure(figsize=(8,5))
sns.countplot(x='sentiment', data =data)

In [ ]:
pos_tweets = data[data.sentiment == 'Positive']
pos_tweets = pos_tweets.sort_values(['polarity'], ascending= False)
pos_tweets.head()

In [ ]:
text = ' '.join([word for word in pos_tweets['renderedContent']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in positive tweets', fontsize=19)
plt.show()

In [ ]:
neg_tweets = data[data.sentiment == 'Negative']
neg_tweets = neg_tweets.sort_values(['polarity'], ascending= False)
neg_tweets.head()

In [ ]:
text = ' '.join([word for word in neg_tweets['renderedContent']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in negative tweets', fontsize=19)
plt.show()

In [ ]:
vect = CountVectorizer(ngram_range=(1,2)).fit(data['renderedContent'])

In [ ]:
vect = CountVectorizer(ngram_range=(1,2)).fit(data['renderedContent'])
feature_names = vect.get_feature_names()
print("Number of features: {}\n".format(len(feature_names)))
print("First 20 features:\n {}".format(feature_names[:20]))

In [ ]:
X = data['renderedContent']
Y = data['sentiment']
X = vect.transform(X)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

In [ ]:
style.use('classic')
cm = confusion_matrix(y_test, logreg_pred, labels=logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels=logreg.classes_)
disp.plot()


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={'C':[0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid)
grid.fit(x_train, y_train)

In [ ]:
print("Best parameters:", grid.best_params_)

In [ ]:
y_pred = grid.predict(x_test)

In [ ]:
logreg_acc = accuracy_score(y_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)

In [ ]:
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("test accuracy: {:.2f}%".format(svc_acc*100))

In [ ]:
print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

In [ ]:
grid = {
    'C':[0.01, 0.1, 1, 10],
    'kernel':["linear","poly","rbf","sigmoid"],
    'degree':[1,3,5,7],
    'gamma':[0.01,1]
}
grid = GridSearchCV(SVCmodel, param_grid)
grid.fit(x_train, y_train)

In [ ]:
print("Best parameter:", grid.best_params_)

In [ ]:
y_pred = grid.predict(x_test)

In [ ]:
logreg_acc = accuracy_score(y_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))